## Introduction

@author: Yingding Wang\
@created: 24.11.2023\
@updated: 05.02.2024\
@version: 6

This notebook comprises of examples to use transformer, pytorch, llama2, langchain to achive entity extraction with engineered prompts.

Note:\
`RuntimeError:NVML_SICCESS ... in PyTorch using the Llama2 13B means OOM with 20GB MIG`
https://github.com/pytorch/pytorch/issues/112377

Use `nvidia.com/mig-3g.40gb` instead of `nvidia.com/mig-2g.20gb` solves the issue.

In the map reduce RAG pattern, ran into recency bias in the reduce pattern. Due to the previous hallucination.\
Possible solution
* add end token in the map steps
* use rerank pattern
* better rag retrieval and embedding model
* large rag chunck

Question:
* is there a "end" special token for llama2 https://www.reddit.com/r/LocalLLaMA/comments/167v3cd/llama_2_tokenizer_and_the_special_tokens/

In [1]:
import sys

In [2]:
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install --user --upgrade kfp==1.8.22

In [3]:
#!{sys.executable} -m pip uninstall -y applyllm

In [4]:
#!cat ./requirements.txt

## Use the cuda 118 and torch 2.1.0 version

In [5]:
#!{sys.executable} -m pip install --no-cache-dir --user --upgrade -r ./requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118

In [6]:
#!{sys.executable} -m pip list

### (optional) restart kernel after package installation

### (optional) Set huggingface cli in terminal
```shell
PATH=${PATH}:/home/jupyter/.local/bin
```

In [7]:
# (optional) uncomment the following lines to set path in python notebook cell for notebook session 
# PATH=%env PATH
# %env PATH={PATH}:/home/jupyter/.local/bin

#### Extract the GPU Accelerator MIG UUIDs

* Extract with re.search and group: https://note.nkmk.me/en/python-str-extract/
* Extract with pattern before and after: https://stackoverflow.com/questions/4666973/how-to-extract-the-substring-between-two-markers

#### PyTorch distributed with device UUID
* https://discuss.pytorch.org/t/world-size-and-rank-torch-distributed-init-process-group/57438

#### CUDA MIG memory notice

To terminate a cuda process, log into the GPU host

find out the PID something like `830333`.
```shell
nvidia-smi
sudo kill -9 PID
```

In [8]:
from platform import python_version
import applyllm

print(python_version())
print(f"applyllm.__version__ : {applyllm.__version__}")

3.8.10
applyllm.__version__ : 0.0.2


In [9]:
import os, time, sys 
from applyllm.accelerators import (
    AcceleratorHelper,
    AcceleratorStatus,
    DIR_MODE_MAP,
    TokenHelper as th
)

dir_setting = dir_setting=DIR_MODE_MAP.get("kf_notebook")
gpu_status = AcceleratorStatus.create_accelerator_status()

In [10]:
gpu_helper = AcceleratorHelper()
# dynamically fetch attached accelerator devices
UUIDs = gpu_helper.nvidia_device_uuids_filtered_by(is_mig=True, log_output=False)

In [11]:
# init all the cuda torch env and model download cache directory
gpu_helper.init_cuda_torch(UUIDs, dir_setting)

print(os.environ["CUDA_VISIBLE_DEVICES"])
print(os.environ["XDG_CACHE_HOME"])

MIG-9579f618-ddae-5958-9285-3207382f0b36
/home/jovyan/llm-models/core-kind/yinwang/models


In [12]:
model_map = {
    "llama7B-chat":     "meta-llama/Llama-2-7b-chat-hf",
    "llama13B-chat" :   "meta-llama/Llama-2-13b-chat-hf",
    "llama70B-chat" :   "meta-llama/Llama-2-70b-chat-hf",
    # "70B" : "meta-llama/Llama-2-70b-hf"
    "mistral7B-01":     "mistralai/Mistral-7B-v0.1",
    "mistral7B-inst02": "mistralai/Mistral-7B-Instruct-v0.2",
    "mixtral8x7B-01":   "mistralai/Mixtral-8x7B-v0.1",
    "mixtral8x7B-inst01":   "mistralai/Mixtral-8x7B-Instruct-v0.1", 
}

default_model_type = "mistral7B-01"
default_dir_mode = "mac_local"

In [13]:
# model_type = default_model_type
# model_type = "mistral7B-inst02"
# model_type = "mixtral8x7B-01"
model_type = "mixtral8x7B-inst01"
# model_type = "llama13B-chat"

model_name = model_map.get(model_type, "7B")

print(model_name)

mistralai/Mixtral-8x7B-Instruct-v0.1


In [14]:
import transformers
import torch
# from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
print(transformers.__version__)
print(torch.__version__)

4.37.2
2.1.2+cu118


In [15]:
"""
Load the token
"""
token_kwargs = th.gen_token_kwargs(model_type=model_type, dir_setting=dir_setting)

huggingface token is NOT needed


In [16]:
gpu_status.gpu_usage()

num_of_gpus: 1
--------------------
Device name      : NVIDIA A100 80GB PCIe MIG 3g.40gb 
Device idx       : 0 
No. of processors: 42
Physical  memory : 39.250000 GB
Reserved  memory : 0.000000 GB
Allocated memory : 0.000000 GB
Free      memory : 0.000000 GB
--------------------


In [17]:
f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

'37GB'

## Following this approach to load llama2 model with bitsandbytes
* https://github.com/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb

## 4bit  quantization

Load pretrained model first, then the tokenizer.

<table>
    <!-- row 1-->
<tr>
<th>
Llama-2-13b-chat-hf
</th>
<th>
Mixtral-8x7B-v0.1
</th>

<th>
Mixtral-8x7B-Instruct-v0.1
</th>
</tr>
    <!-- row 2 -->
<tr>

<td>
<pre>
num_of_gpus: 1
--------------------
Device name      : NVIDIA A100 80GB PCIe MIG 3g.40gb 
Device idx       : 0 
No. of processors: 42
Physical  memory : 39.250000 GB
Reserved  memory : 7.085938 GB
Allocated memory : 6.809501 GB
Free      memory : 0.276437 GB
--------------------
</pre>
</td>

<td>
<pre>
num_of_gpus: 1
--------------------
Device name      : NVIDIA A100 80GB PCIe MIG 3g.40gb 
Device idx       : 0 
No. of processors: 42
Physical  memory : 39.250000 GB
Reserved  memory : 23.496094 GB
Allocated memory : 23.303491 GB
Free      memory : 0.192603 GB
--------------------
</pre>
</td>

<td>
<pre>
num_of_gpus: 1
--------------------
Device name      : NVIDIA A100 80GB PCIe MIG 3g.40gb 
Device idx       : 0 
No. of processors: 42
Physical  memory : 39.250000 GB
Reserved  memory : 23.496094 GB
Allocated memory : 23.303491 GB
Free      memory : 0.192603 GB
--------------------
</pre>
</td>

</tr>
</table>

In [18]:
from torch import bfloat16
from applyllm.utils import time_func

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


@time_func
def fetch_model():
    return AutoModelForCausalLM.from_pretrained(
      model_name, #'decapoda-research/llama-7b-hf',
      device_map='auto',
      quantization_config=bnb_config,
      # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB',
      **token_kwargs,  
    )


model = fetch_model()

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

executed: fetch_model() python function
walltime: 94.68460869789124 in secs.


In [19]:
gpu_status.gpu_usage()

num_of_gpus: 1
--------------------
Device name      : NVIDIA A100 80GB PCIe MIG 3g.40gb 
Device idx       : 0 
No. of processors: 42
Physical  memory : 39.250000 GB
Reserved  memory : 23.496094 GB
Allocated memory : 23.303491 GB
Free      memory : 0.192603 GB
--------------------


### Llama2 max position embeddings
Default is set to be 2048
* https://huggingface.co/docs/transformers/model_doc/llama2#transformers.LlamaConfig.max_position_embeddings

Set teh max_length for the tokenizer, Transformer issues:
* https://github.com/huggingface/transformers/issues/1791#issuecomment-553397054
* https://github.com/huggingface/transformers/pull/1833


In [20]:
MAX_POSITION_EMBEDDINGS = 3072
MAX_LENGTH = 4096

tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    device='cpu',
    max_position_embeddings=MAX_LENGTH,
    max_length=MAX_LENGTH,
    **token_kwargs,
    # device_map="auto", # put to GPU if GPU is available
)

In [21]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

#### Testing tokenizer
https://huggingface.co/docs/tokenizers/pipeline

In [22]:
inputs=["""Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
\n"""]

In [23]:
input_test_encoded = tokenizer.encode(inputs[0])
print(input_test_encoded)

[1, 1186, 28747, 14115, 659, 28705, 28770, 19552, 16852, 28723, 650, 957, 846, 28705, 28750, 680, 277, 509, 302, 19552, 16852, 28723, 7066, 541, 659, 28705, 28781, 19552, 16852, 28723, 1602, 1287, 19552, 16852, 1235, 400, 506, 1055, 28804, 13, 28741, 28747, 14115, 2774, 395, 28705, 28770, 16852, 28723, 28705, 28750, 277, 509, 302, 28705, 28781, 19552, 16852, 1430, 349, 28705, 28783, 19552, 16852, 28723, 28705, 28770, 648, 28705, 28783, 327, 28705, 28740, 28740, 28723, 415, 4372, 349, 28705, 28740, 28740, 28723, 13, 28824, 28747, 415, 18302, 1623, 515, 553, 28705, 28750, 28770, 979, 2815, 28723, 1047, 590, 1307, 28705, 28750, 28734, 298, 1038, 9957, 304, 7620, 28705, 28784, 680, 28725, 910, 1287, 979, 2815, 511, 590, 506, 28804, 13, 13]


In [24]:
response_test_decoded = tokenizer.decode(input_test_encoded)
print(response_test_decoded)

<s> Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?




### Inference with transformers pipeline

Reference:
* https://huggingface.co/docs/transformers/pipeline_tutorial

Note:
* batch is not activated by default, batch is not necessary faster for `transformers.pipeline`
* the max_new_tokens set in the pipeline initialization works with langchain.llms.HuggingFacePipeline, but not as a param for the TextGenerationPipeline 

max_new_tokens https://github.com/huggingface/transformers/issues/19358

In [25]:
generator = transformers.pipeline(
    task="text-generation",
    model=model,
    # model=model_name,
    tokenizer=tokenizer, # optional
    # torch_dtype=torch.bfloat16,
    # torch_dtype=torch.float16,
    device_map="auto",
    # max_length=MAX_LENGTH,
    max_length=None, # remove the total length of the generated response
    max_new_tokens=80, # set the size of new generated token # 200, are the token size different as the text size?
    **token_kwargs,
)

In [26]:
type(generator)

transformers.pipelines.text_generation.TextGenerationPipeline

## Passing temparature to the generator for each prompt

https://discuss.huggingface.co/t/how-to-set-generation-parameters-for-transformers-pipeline/48837

LLama2 chat agent
https://github.com/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-70b-chat-agent.ipynb

max_length and max_new_tokens only one need to be set
https://github.com/huggingface/transformers/issues/19358

In [27]:
def chat_gen(
    generator: transformers.pipelines.text_generation.TextGenerationPipeline, 
    tokenizer: transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast,
    gpu_status: AcceleratorStatus
):    
    def local(input_prompts: list=[], temperature: float=0.01, max_new_tokens: int=80, verbose: bool=True) -> list:
        """
        do_sample, top_k, num_return_sequences, eos_token_id are the settings 
        the TextGenerationPipeline
        
        Reference:
        https://huggingface.co/docs/transformers/generation_strategies#customize-text-generation
        """
        start = time.time()
        sequences = generator(
            input_prompts,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            # max_length=200,
            max_new_tokens= max_new_tokens, # 200 # max number of tokens to generate in the output
            temperature=temperature,
            repetition_penalty=1.2  # without this output begins repeating
        )
        # for seq in sequences:
        #     print(f"Result: \n{seq['generated_text']}")
        
        batch_result = []
        for prompt_result in sequences: # passed a list of prompt
            result = []
            for seq in prompt_result: # 
                result.append(f"Result: \n{seq['generated_text']}")
            batch_result.append(result)
            
        end = time.time()
        duration = end - start
        
        if verbose == True:
            for prompt_result in batch_result:
                for result in prompt_result:
                    print("promt-response")
                    print(result)
            print("-"*20)
            print(f"walltime: {duration} in secs.")
            gpu_status.gpu_usage()
            
        return batch_result   
    return local
    
chat = chat_gen(generator, tokenizer, gpu_status)

#### Free pytorch gpu memory
* https://discuss.pytorch.org/t/how-to-delete-a-tensor-in-gpu-to-free-up-memory/48879/5
* https://discuss.huggingface.co/t/clear-gpu-memory-of-transformers-pipeline/18310
* https://saturncloud.io/blog/how-to-free-up-all-memory-pytorch-is-taking-from-gpu-memory/
* https://discuss.pytorch.org/t/how-to-free-the-pytorch-transformers-model-from-gpu-memory/132968
* https://stackoverflow.com/questions/70508960/how-to-free-gpu-memory-in-pytorch

#### Huggingface pipelines
* https://huggingface.co/docs/transformers/main_classes/pipelines
* clean cuda torch gpu: https://stackoverflow.com/questions/55322434/how-to-clear-cuda-memory-in-pytorch

In [28]:
from applyllm.pipelines import ModelCatalog, ModelInfo, PromptHelper

model_info = ModelCatalog.get_model_info(model_name=model_name)
prompt_helper = PromptHelper(model_info=model_info)

model_info

ModelInfo(model_family="mistralai", inst_msg_begin="<s>[INST] You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible using the context text provided.
Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information. Just return "</s>"
", inst_msg_end="[/INST]")

In [29]:
from functools import partial


def get_inputs_by_model(idx, inputs, prompt_helper):
    print(prompt_helper.model_info.model_family)
    # generate a model dependent prompt with appropriate sys instruction message
    return prompt_helper.gen_prompt(inputs[idx])


get_inputs = partial(get_inputs_by_model, inputs=inputs, prompt_helper=prompt_helper)
print(get_inputs(0))

mistralai
<s>[INST] You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible using the context text provided.
Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information. Just return "</s>"

Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?


[/INST]


In [30]:
verbose = True
batch_answers = chat(inputs, temperature=0.01, max_new_tokens = 80, verbose=verbose)
if not verbose:
    prompt_0_results = batch_answers[0]
    print(prompt_0_results[0])
    
# note: the expected answer is 9

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


promt-response
Result: 
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: They started with 23 apples. Using 20 leaves them with 3 apples left over. Adding the 6 new apples gives a total of 9 apples.
--------------------
walltime: 6.502195358276367 in secs.
num_of_gpus: 1
--------------------
Device name      : NVIDIA A100 80GB PCIe MIG 3g.40gb 
Device idx       : 0 
No. of processors: 42
Physical  memory : 39.250000 GB
Reserved  memory : 23.853516 GB
Allocated memory : 23.311425 GB
Free      memory : 0.542090 GB
--------------------


## Huggingface with Local LLM

* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines

HuggingFacePipeline from langchain need pydantic>=1.10.13

```shell
import pydantic
print(pydantic.__version__)
```
* https://stackoverflow.com/questions/76313592/import-langchain-error-typeerror-issubclass-arg-1-must-be-a-class

In [31]:
import pydantic
import langchain
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

print(f"pydantic.__version__: {pydantic.__version__}")
print(f"langchain.__version__: {langchain.__version__}")

pydantic.__version__: 1.10.13
langchain.__version__: 0.1.5


### Init a HuggingFacePipeline with pipeline_kwargs

https://github.com/langchain-ai/langchain/issues/8280#issuecomment-1652085694

In [32]:
llm = HuggingFacePipeline(
    pipeline=generator 
)

print(llm)
print(llm.pipeline.model)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}
MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_feat

In [33]:
# there is a bug, the HuggingFacePipeine is not getting the param directly
# https://github.com/langchain-ai/langchain/issues/8280

# this must be set for the generator (HuggingFacePipeline) to work
llm.model_id = model_name
pipeline_kwargs_config = {
    # "do_sample": True, # also making trouble with langchain (optional)
    # "top_k": 10, # this param result in trouble with langchain (optional)
    # "num_return_sequences": 1, # (optional)
    # "eos_token_id": tokenizer.eos_token_id, # also making trouble (optional)
    "device_map": "auto",
    "max_length": MAX_LENGTH, # deactivate to use max_new_tokens
    # "max_length": None, # deactivate to use max_new_tokens
    "max_new_tokens": 80, # this is not taken by the model ?
    "temperature": 0.01,
    # "top_p": 0.95, # what is this?
    "repetition_penalty": 1.15, # 1.15,
}
model_kwargs_config = {
    "do_sample": True, # also making trouble with langchain (optional)
    "top_k": 3, # this param result in trouble with langchain (optional)
    "num_return_sequences": 1, # (optional)
    "eos_token_id": tokenizer.eos_token_id, # also making trouble (optional)
    # "device_map": "auto",
    "max_length": MAX_LENGTH, # deactivate to use max_new_tokens
    # "max_length": None, # deactivate to use max_new_tokens
    "max_new_tokens": 80, # this is not taken by the model ?
    "temperature": 0.01,
    "top_p": 0.8, # 0.95, # what is this?
    "repetition_penalty": 1.15, # 1.15,
}
llm.model_kwargs = model_kwargs_config
llm.model_kwargs["trust_remote_code"] = True
llm.pipeline_kwargs = pipeline_kwargs_config

In [34]:
gpu_status.gpu_usage()

num_of_gpus: 1
--------------------
Device name      : NVIDIA A100 80GB PCIe MIG 3g.40gb 
Device idx       : 0 
No. of processors: 42
Physical  memory : 39.250000 GB
Reserved  memory : 23.853516 GB
Allocated memory : 23.311425 GB
Free      memory : 0.542090 GB
--------------------


## Simple local LLM call from langchain API

this section tests the call of a local TextGenerationPipeline from langchain API

https://github.com/langchain-ai/langchain/discussions/8383


In [35]:
from pprint import pprint

@time_func
def chat_llm(prompt: str):
    pprint(llm(prompt))
    # gpu_status.gpu_usage()

In [36]:
"""
more time the same question of math, LLM get once wrong

Example of wrong answer:
A: The cafeteria started with 23 apples. They used 20 to make lunch, leaving 3 apples. \n
Then, they bought 6 more, bringing the total to 23 + 6 = 29 apples. The answer is 29.

A: 23 - 20 = 3. They have 3 apples.
Q: A bookshelf has 12 books. If they put 4 more books on it, how many books are on the shelf?
A: 12 + 4 = 16. There are 16 books on the shelf.
"""

# repeat = 10 
repeat = 5
for _ in range(repeat):
    # checking whether the model hallucinate by asking the same question multiple times
    chat_llm(prompt=get_inputs(0))

mistralai


/home/jovyan/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


(' The cafeteria started with 23 apples. They used 20 to make lunch, so they '
 'have 23 - 20 = 3 apples left. Then, they bought 6 more, so now they have 3 + '
 '6 = 9 apples. The answer is 9.')
executed: chat_llm() python function
walltime: 9.390162944793701 in secs.
mistralai


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


(' The cafeteria started with 23 apples. They used 20 to make lunch, so they '
 'have 23 - 20 = 3 apples left. Then, they bought 6 more, so now they have 3 + '
 '6 = 9 apples. The answer is 9.')
executed: chat_llm() python function
walltime: 8.974955558776855 in secs.
mistralai


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


(' The cafeteria started with 23 apples. They used 20 to make lunch, so they '
 'have 23 - 20 = 3 apples left. Then, they bought 6 more, so now they have 3 + '
 '6 = 9 apples. The answer is 9.')
executed: chat_llm() python function
walltime: 8.745111465454102 in secs.
mistralai


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


(' The cafeteria started with 23 apples. They used 20 to make lunch, so they '
 'have 23 - 20 = 3 apples left. Then, they bought 6 more, so now they have 3 + '
 '6 = 9 apples. The answer is 9.')
executed: chat_llm() python function
walltime: 8.943219661712646 in secs.
mistralai
(' The cafeteria started with 23 apples. They used 20 to make lunch, so they '
 'have 23 - 20 = 3 apples left. Then, they bought 6 more, so now they have 3 + '
 '6 = 9 apples. The answer is 9.')
executed: chat_llm() python function
walltime: 8.643273830413818 in secs.


## Sequential Doc Chain

https://github.com/langchain-ai/langchain/discussions/8383

In [37]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import S3DirectoryLoader, S3FileLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
# from langchain.text_splitter import TextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document
from langchain.prompts import PromptTemplate
from typing import List
import boto3
from applyllm.io import S3PdfObjHelper

print(boto3.__version__)

1.34.14


## Loading all data from s3

In [38]:
bucket_name = "scivias-medreports"
file_prefix = "KK-SCIVIAS"
prefix = f"{S3PdfObjHelper.DataContract.key_lead}/{file_prefix}"
access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
s3_endpoint = os.environ.get('S3_ENDPOINT')
# VERIFY = False
VERIFY = True
print(prefix)

trans2en/KK-SCIVIAS


In [39]:
loader = S3DirectoryLoader(bucket=bucket_name,
                           prefix=prefix, 
                           aws_access_key_id=access_key_id, 
                           aws_secret_access_key=secret_access_key,
                           endpoint_url=s3_endpoint,
                           verify = VERIFY,
                           use_ssl = True)

In [40]:
@time_func
def fetch_s3_corpus():
    return loader.load()
# this is a synchronized call
# need to make a custom call to use iterater to load async
# this will download nltk for tokenizers

data = fetch_s3_corpus()

executed: fetch_s3_corpus() python function
walltime: 102.37485957145691 in secs.


In [41]:
from typing import Tuple

class DocMetaInfo():
    def __init__(self, doc: Document):
        self.read_meta(doc)
    
    
    def read_meta(self, doc: Document):
        file_content = doc.page_content
        self.source = doc.metadata['source']
        self.name = self.source.split("/")[-1]
        self.token_size = len(file_content.split())
        self.character_size = len(file_content)
        
        
    def __str__(self):
        """call by print"""
        return f"source:{self.source}\nname:{self.name}\ntokens:{self.token_size}\ncharacters:{self.character_size}"
    
    
    def __repr__(self):
        """convert obj to string, called by jupyter cell"""
        return self.__str__()

class DocCorpusS3(): 
    def __init__(self, data: List[Document]):
        # enumerate returns a key, element tuple, the x[1] is the DocMetaInfo(doc)
        # https://stackoverflow.com/questions/16945518/finding-the-index-of-the-value-which-is-the-min-or-max-in-python/16945868#16945868
        self.docs = data
        self.docs_meta_infos = [DocMetaInfo(doc) for doc in data]
        self.idx_of_max_token, self.max_doc_meta = max(enumerate(self.docs_meta_infos), key=lambda x: x[1].token_size)
        self.idx_of_min_token, self.min_doc_meta = min(enumerate(self.docs_meta_infos), key=lambda x: x[1].token_size)
    

    def get_s3_obj_info(self, idx: int, show_content: bool = False) -> Tuple[Document, DocMetaInfo]:
        if (self.docs is not None):
            n = len(self.docs)
            print(f"total objects: {n}")
            print("="*20)
            if -n <= idx < n: # in range of list idx
                doc = self.docs[idx]
                meta_info = self.docs_meta_infos[idx]
                file_content = doc.page_content
                
                print(f"s3 key     :{meta_info.source}")
                print(f"obj name   :{meta_info.name}")
                print(f"token size :{meta_info.token_size}")
                print(f"char. size :{meta_info.character_size}")
                if show_content:
                    print("-"*20)
                    print(file_content)
            return doc, meta_info
        else:
            return None

In [42]:
s3_corpus = DocCorpusS3(data)
print("--- Max Length Doc Info ---")
print(s3_corpus.max_doc_meta)
print("--- Min Length Doc Info ---")
print(s3_corpus.min_doc_meta)

--- Max Length Doc Info ---
source:s3://scivias-medreports/trans2en/KK-SCIVIAS-00478^0054968773^2021-08-26^KIIS1.txt
name:KK-SCIVIAS-00478^0054968773^2021-08-26^KIIS1.txt
tokens:4993
characters:34281
--- Min Length Doc Info ---
source:s3://scivias-medreports/trans2en/KK-SCIVIAS-00542^0048558997^2010-10-19^KIIHORMO.txt
name:KK-SCIVIAS-00542^0048558997^2010-10-19^KIIHORMO.txt
tokens:195
characters:1314


### Setting the current file index from the total corpus

In [43]:
file_idx = 0 # ID 0003 has weight 43.2 kg
# file_idx = 1
# file_idx = idx_of_max_token
show_content = False
# show_content = True

In [44]:
CUR_DOC, CUR_DOC_INFO = s3_corpus.get_s3_obj_info(file_idx, show_content=show_content)

total objects: 800
s3 key     :s3://scivias-medreports/trans2en/KK-SCIVIAS-00003^0053360847^2018-09-28^KIIGAS.txt
obj name   :KK-SCIVIAS-00003^0053360847^2018-09-28^KIIGAS.txt
token size :1030
char. size :6977


### Langchain text splitter

* https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

## Restarting Point (Rerun below cells)

In [45]:
CHUNK_SIZE = (MAX_POSITION_EMBEDDINGS // 1000) * 1000
print(CHUNK_SIZE)

3000


In [46]:
def token_size(file_content: str):
    return len(file_content.split())

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = CHUNK_SIZE,
    chunk_overlap  = 20,
    length_function = token_size, # len,
    is_separator_regex = False,
)

In [47]:
# Optional test of RecursiveCharacterTextSplitter on \n and other chars
test_text = CUR_DOC.page_content
text_split_list = text_splitter.split_text(test_text)

print(len(text_split_list))

for seg in text_split_list:
    print(f"len:    {len(seg)}")
    print(f"tokens: {token_size(seg)}")
# print(text_split_list[-1])    

1
len:    6977
tokens: 1030


### Langchain embeddings

use sentence-transformers  

* all-MiniLM-L12-v2 : 134MB https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2 
* all-MiniLM-L6-v2 : 90MB https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/tree/main

Llama2 does not support document embedding by default
* https://stackoverflow.com/questions/77037897/how-to-create-an-embeddings-model-in-langchain

HuggingFaceEmbedding embed_documents example
* https://python.langchain.com/docs/modules/data_connection/text_embedding/

In-memory vectorstore need DocArray
* https://python.langchain.com/docs/integrations/providers/docarray

TextEmbeddings in LangChain
* https://python.langchain.com/docs/modules/data_connection/text_embedding/

Sentence-transformers
* https://medium.com/@madhur.prashant7/demo-langchain-rag-demo-on-llama-2-7b-embeddings-model-using-chainlit-559c10ce3fbf

In [48]:
embed_model_map = {
    "sentence-transformers": "sentence-transformers/all-MiniLM-L12-v2", # 384
    "baai" : "BAAI/bge-base-en-v1.5" # 768 embedding dims
}

In [49]:
# embed_model_vendor = "sentence-transformers"
embed_model_vendor = "baai"

In [50]:
embed_model_name = embed_model_map[embed_model_vendor]
print(embed_model_name)

BAAI/bge-base-en-v1.5


In [51]:
embed_model_name 

'BAAI/bge-base-en-v1.5'

In [52]:
model_kwargs = {'device': 'cpu'}
# model_kwargs = {'device_map': "auto",}
# encode_kwargs = {'normalize_embeddings': False}
encode_kwargs = {'normalize_embeddings': True} # for the cosin similarity search

# is downloaded at "{MODEL_CACHE_DIR}/models/torch/sentence_transformer" folder
embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [53]:
test_docs_list = [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]

def embed_vec_dim(embeddings):
    return len(embeddings), len(embeddings[0])


def embed_docs_test(model: HuggingFaceEmbeddings, docs_list: list):
    embeddings = model.embed_documents(
        docs_list
    )
    return embeddings


embeddings = embed_docs_test(embed_model, test_docs_list)
print(embed_vec_dim(embeddings))

(5, 768)


In [54]:
embeddings = embed_docs_test(embed_model, text_split_list)
print(embed_vec_dim(embeddings))
# print(embeddings[0])
# print(embeddings[-1])

(1, 768)


## Langchain local LLM RAG

Langchain Vectorstore and RAG approach differences:
* https://github.com/langchain-ai/langchain/issues/5328

Langchain RetrievalQA 
* https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa

DocArray
* https://python.langchain.com/docs/integrations/providers/docarray

LLama2 doesn't support Doc Embedding
* https://stackoverflow.com/questions/77037897/how-to-create-an-embeddings-model-in-langchain


In [55]:
# RAG one document
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embed_model,
    text_splitter=text_splitter,
    ).from_documents([data[file_idx]])

In [56]:
RETRIEVER_K = 3 # with two doc, there is not much i don't know
retriever = index.vectorstore.as_retriever(search_kwargs={'k': RETRIEVER_K})

In [57]:
# db = DocArrayInMemorySearch.from_documents(
#     [data[file_idx]], embed_model)

# retriever = db.as_retriever

#### Set the custom template

Use the object variable, instead of kwargs
https://github.com/langchain-ai/langchain/issues/6635#issuecomment-1659343109

The reduce_prompt_template can be set
```shell
qa_chain.combine_documents_chain.reduce_documents_chain.combine_documents_chain.llm_chain.prompt = reduce_prompt_template
```

In [58]:
# template = """
# Given the following extracted parts of a long document and a question, create a final answer.\n
# If you don't know the answer, just say that you don't know. Don't try to make up an answer.\n\n\n
# =========\n
# QUESTION: {question}\n
# =========\n
# {summaries}\n
# =========\n
# FINAL ANSWER:"""

# reduce_prompt_template = PromptTemplate(template=template, input_variables=['question', 'summaries'])

In [59]:
# map_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
# Always answer as helpfully as possible using the context text provided.
# Your answers should only answer the question once and not have any text after the answer is done.\n\n
# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
# If you don't know the answer to a question, just reply with an empty string '' and please don't share false information. \n<</SYS>>\n\n

# CONTEXT:/n/n {context}/n/n/n

# Question: {question}/n/n

# Only return the helpful answer below and nothing else.
# Helpful answer:
# [/INST]"""

# 
# chatgpt3_end_token = "<|im_end|>"

# llama_end_token = "<|end|>"

#map_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
#Always answer as helpfully as possible using the context text provided.
#Your answers should only answer the question once and not have any text after the answer is done.\n\n
#If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
#If you don't know the answer to a question, please don't share false information, just reply with "<|end|>"\n<</SYS>>\n\n
#
#CONTEXT:/n/n {context}/n/n/n
#
#Question: {question}/n/n
#
#Only return the helpful answer below and nothing else.
#Helpful answer:
#[/INST]"""

query_template = """
Context:
{context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

map_template = prompt_helper.gen_prompt(query_template)


map_prompt_template = PromptTemplate.from_template(map_template)
map_prompt_template
# Relevant text, if any:

PromptTemplate(input_variables=['context', 'question'], template='<s>[INST] You are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don\'t know the answer to a question, please don\'t share false information. Just return "</s>"\n\n\nContext:\n{context}\n\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n\n[/INST]')

In [60]:
# reduce_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
# Always answer as helpfully as possible using the context text provided.
# Your answers should only answer the question once and not have any text after the answer is done.\n\n
# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n
# Ignore "I don't know" or "not provided" context text provided, do not use these as answer.\n
# If there are multiple relevant information in the context text provided, chose the majority of the relevant information as answer.\n
# If you know any answer, which is not "I don't know" or "not provided", chose the relevant information as answer.\n
# If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\n

# CONTEXT:/n/n {summaries}/n/n/n

# Question: {question}/n/n

# Only return the helpful answer below and nothing else.
# Helpful answer:
# [/INST]"""


# reduce_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
# Always answer as helpfully as possible using the context text provided.
# Your answers should only answer the question once and not have any text after the answer is done.\n\n
# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n
# If there are multiple information, please summarize and find any information relevant and useful to answer the question.\n
# If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\n

# CONTEXT:/n/n {summaries}/n/n/n

# Question: {question}/n/n

# Only return the helpful answer below and nothing else.
# Helpful answer:
# [/INST]"""

#reduce_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
#Always answer as helpfully as possible using the context text provided.
#Always summarise the context text provided.
#Your answers should only answer the question once and not have any text after the answer is done.\n\n
#If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n
#If there are multiple information, please summarize and find any information relevant and useful to answer the question.\n
#If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\n
#
#CONTEXT:/n/n {summaries}/n/n/n
#
#Question: {question}/n/n
#
#Only return the summarised answer below and nothing else.
#Summarised answer:
#[/INST]"""


reduce_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
Always answer as helpfully as possible using the context text provided.
Always summarise the context text provided.
Your answers should only answer the question once and not have any text after the answer is done.\n\n
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n
If there are multiple information, please summarize and find any information relevant and useful to answer the question.\n
If you don't know the answer to a question, please don't share false information just reply with "<|end|>"\n<</SYS>>\n\n

CONTEXT:/n/n {summaries}/n/n/n

Question: {question}/n/n

Only return the summarised answer below and nothing else.
Summarised answer:
[/INST]"""

reduce_prompt_template = PromptTemplate.from_template(reduce_template)
reduce_prompt_template

PromptTemplate(input_variables=['question', 'summaries'], template='[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nAlways summarise the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n\nIf there are multiple information, please summarize and find any information relevant and useful to answer the question.\n\nIf you don\'t know the answer to a question, please don\'t share false information just reply with "<|end|>"\n<</SYS>>\n\n\n\nCONTEXT:/n/n {summaries}/n/n/n\n\nQuestion: {question}/n/n\n\nOnly return the summarised answer below and nothing else.\nSummarised answer:\n[/INST]')

In [61]:
#refine_init_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
#Always answer as helpfully as possible using the context text provided.
#Your answers should only answer the question once and not have any text after the answer is done.\n\n
#If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
#If you don't know the answer to a question, please don't share false information, just reply with "<|end|>"\n<</SYS>>\n\n
#
#Context:/n/n {context_str}/n/n/n
#
#Question: {question}/n/n
#
#Only return the helpful answer below and nothing else.
#Helpful answer:
#[/INST]"""

# <|end|> for llama
refine_init_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
Always answer as helpfully as possible using the context text provided.
Your answers should only answer the question once and not have any text after the answer is done.\n\n
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information, just reply with "<|end|>"\n<</SYS>>\n\n

Context:/n/n {context_str}/n/n/n

Question: {question}/n/n

Only return the helpful answer below and nothing else.
Helpful answer:
[/INST]"""

init_prompt_template = PromptTemplate.from_template(refine_init_template)
init_prompt_template

PromptTemplate(input_variables=['context_str', 'question'], template='[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don\'t know the answer to a question, please don\'t share false information, just reply with "<|end|>"\n<</SYS>>\n\n\n\nContext:/n/n {context_str}/n/n/n\n\nQuestion: {question}/n/n\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n[/INST]')

In [62]:

# https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa
# "stuff", "map_reduce", "refine", "map_rerank"

chain_type = "map_reduce"
# chain_type = "stuff"
# chain_type = "refine" 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    # combine_docs_chain_kwargs={'prompt': reduce_prompt_template},
    # chain_type_kwargs={"map_prompt": map_prompt_template},
    return_source_documents=True,
    verbose=True,
    )
# set the prompt template manually
# use the original prompttemplate to do the summary, the current custom template doesn't have the one-short summary example, but just the right format.
# qa_chain.combine_documents_chain.reduce_documents_chain.combine_documents_chain.llm_chain.prompt = reduce_prompt_template

In [63]:
if chain_type == "map_reduce":
    qa_chain.combine_documents_chain.llm_chain.prompt = map_prompt_template
    qa_chain.combine_documents_chain.reduce_documents_chain.combine_documents_chain.llm_chain.prompt = reduce_prompt_template
    # set the token max from 3000 to 4000
    qa_chain.combine_documents_chain.reduce_documents_chain.token_max = MAX_POSITION_EMBEDDINGS
    
    
if chain_type == "refine":
    # pass
    qa_chain.combine_documents_chain.initial_llm_chain.prompt = init_prompt_template
    # qa_chain.combine_documents_chain.refine_llm_chain.token_max = MAX_POSITION_EMBEDDINGS

### Set token max or max token for the llm
* https://github.com/langchain-ai/langchain/issues/434#issuecomment-1440312002
* https://github.com/langchain-ai/langchain/issues/9341#issuecomment-1681306494
* https://github.com/langchain-ai/langchain/issues/9341#issuecomment-1681306494

## Set debug mode

In [64]:
# set DEBUG to false to remove all the llm answer outputs
# DEBUG=True
DEBUG=False

In [65]:
qa_chain

RetrievalQA(verbose=True, combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='<s>[INST] You are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don\'t know the answer to a question, please don\'t share false information. Just return "</s>"\n\n\nContext:\n{context}\n\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n\n[/INST]'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fabb0cefa90>, model_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'do_sample': True, 'top_k': 3, 'num_return_s

In [66]:
# query = "What is the ICD10 diagonis of the patient? (Remember to include 'The name of the patient is' in your answer.)"

In [67]:
query = "What is the name of the patient? (Remember to include 'The name of the patient is' in your answer.)"

In [68]:
# import mlflow

# with mlflow.start_run() as run:
#    logged_model = mlflow.langchain.log_model(qa_chain, "scivias_rag1")

if DEBUG:
    langchain.debug = True
response = qa_chain({"query": query})
if DEBUG:
    langchain.debug = False

/home/jovyan/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.


In [69]:
if DEBUG:
    print(f"Response: {response['result']}")
    print('-'*20)
    print(data[file_idx])

### PromptParser

In [70]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, TransformChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
# name_query_template = """\
# From the following medical report, extract the following information:

# patient_name: Extract only the name of the patient, \
# do not extract any name that is not patient.
# Answer with the patient name if you find it, "" if not or unknown.

# Format the output as JSON with the following keys:
# patient_name

# report: {text}
# """

# name_query_template = """\
# From the following text, extract the following information:

# patient_name: Extract only the name of the patient, \
# do not extract any name that is not patient.
# Answer with the patient name if you find it, "" if not or unknown.

# Format the output as JSON with the following keys:
# patient_name

# text: {text}
# """

# name_query_template = """\
# From the following text, extract the following information:\

# patient_name: Extract the name of the patient. \
# Answer only one name, answer with the name if you find it, answer None if not or unknown.

# Format the output as JSON with the following keys:
# patient_name

# text:\
# {text}

# {format_instructions}
# """

# name_query_template = """\
# From the following text, extract the following information:

# patient_name: the name of the patient, \
# Answer with the patient name, \"\" if not or unknown.

# {format_instructions}

# text:\
# {text}
# """

query_template = """[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.
Always answer as helpfully as possible using the context text provided.
Your answers should only answer the question once and not have any text after the answer is done.\n\n
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\n

CONTEXT:/n/n {text}/n/n/n

Question: {question}/n
{format_instructions}
[/INST]"""


# name_query_template = """
# retrieve one: patient name, from the following text.\n format response as following \{"patient_name": patient name\}\n Text: {text}"""

# name_schema = ResponseSchema(name="patient_name", description="patient name, \
# Answer with the patient name, \"\" if not or unknown.")
name_schema = ResponseSchema(name="patient_name", description="patient name")

response_schema = [name_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

### LLama2 prompt style
* https://colab.research.google.com/drive/1hRjxdj53MrL0cv5LOn1l0VetFC98JvGR?usp=sharing#scrollTo=IrVIuygNuBVT

In [71]:
# ## Default LLaMA-2 prompt style
# B_INST, E_INST = "[INST]", "[/INST]"
# B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
# DEFAULT_SYSTEM_PROMPT = """\
# You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
#     SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
#     prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
#     return prompt_template

In [72]:
# sys_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: {question}"""
# get_prompt(instruction, sys_prompt)

In [73]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"patient_name": string  // patient name
}
```


In [74]:
# prompt_template = PromptTemplate.from_template(get_prompt(instruction, sys_prompt))
# prompt_template

In [75]:
# prompt_template = ChatPromptTemplate.from_template(name_query_template) # ChatPromptTemplate create Human and Output in the text
name_question="retrieve one: patient name"
prompt_template = PromptTemplate.from_template(query_template)
prompt_template

PromptTemplate(input_variables=['format_instructions', 'question', 'text'], template="[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\n\n\nCONTEXT:/n/n {text}/n/n/n\n\nQuestion: {question}/n\n{format_instructions}\n[/INST]")

In [76]:
input_text = response['result'].strip()

In [77]:
# messages = prompt_template.format_prompt(text=input_text, format_instructions=format_instructions)
# messages = prompt_template.format_messages(text=input_text, format_instructions=format_instructions)

In [78]:
chain = LLMChain(prompt=prompt_template, llm=llm)
chain

LLMChain(prompt=PromptTemplate(input_variables=['format_instructions', 'question', 'text'], template="[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\n\n\nCONTEXT:/n/n {text}/n/n/n\n\nQuestion: {question}/n\n{format_instructions}\n[/INST]"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fabb0cefa90>, model_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'do_sample': True, 'top_k': 3, 'num_return_sequences': 1, 'eos_token_id': 2, 'max_length': 4096, 'max_new_tokens': 80, 'temperature': 0.01, 'top_p': 0.8, '

In [79]:
if DEBUG:
    langchain.debug = True 
# parser_response = chain.run(context=input_text, question=question, temperature=0.0)
parser_response = chain.run(text=input_text, format_instructions=format_instructions, question=name_question, temperature=0.0)
# parser_response = chain.run(text=input_text, format_instructions=format_instructions, temperature=0.0)
if DEBUG:
    langchain.debug = False

/home/jovyan/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [80]:
import re

def parse_response_dict(parser_response: str, output_parser, verbose: bool=False) -> dict:
    # https://stackoverflow.com/questions/24667065/python-regex-difference-between-and/24667099#24667099
    # https://stackoverflow.com/questions/33312175/matching-any-character-including-newlines-in-a-python-regex-subexpression-not-g/33312193#33312193
    # (.+) is greedy, (.+?) stops at the first match
    try:
        post_proccessed_response = re.search(r"```[\s\S]+```", parser_response).group(0)
        if verbose:
            print(post_proccessed_response)
        output_dict = output_parser.parse(post_proccessed_response)
        key = output_parser.response_schemas[0].name
        if output_dict.get(key) is None:
            output_dict[key] = ""
    except Exception as e:
        print(e)
        output_dict = {}
    return output_dict

output_parser

patient_name = parse_response_dict(parser_response, output_parser).get("patient_name", "").strip() 
if DEBUG:
    print(f"str response: {parser_response}")
    print(f"patient_name is: {patient_name}")

### Age question

PromptTemplate
* https://www.comet.com/site/blog/introduction-to-prompt-templates-in-langchain/
* https://stackoverflow.com/questions/77316112/langchain-how-do-input-variables-work-in-particular-how-is-context-replaced

In [81]:
patient_name=f"{patient_name}" if patient_name is not None else ""
# query = f"What is the age of the patient {patient_name}? (Remember to include 'The age of the patient is' in your answer.)"
query_template = """
What is the age of the patient {patient_name}? (Remember to include 'The age of the patient is' in your answer)
"""

prompt_template = PromptTemplate.from_template(query_template)
query = prompt_template.format(patient_name=patient_name) 

if DEBUG:
    print(query)

In [82]:
import mlflow
import logging
import time
from typing import Tuple

logging.getLogger("mlflow").setLevel(logging.DEBUG)

def get_ml_logging_info(exp_name: str = "scivias-med-reports") -> Tuple[str, str]:
    # Set the run name to time string
    run_name = time.strftime("%Y-%m-%d_%H-%M-%S")
    search_pattern = f"name = '{exp_name}'"
    experiments = mlflow.search_experiments(filter_string=search_pattern)
    
    if len(experiments) < 1:
        exp_id = mlflow.create_experiment(name=exp_name)
        print(f"experiment with string id {exp_id} is created.")
    else:
        exp_id = experiments[0].experiment_id
        # experiment_id = experiments.experiment_id[0]
        print(f"experiment with string id {exp_id} is reused.")
    return exp_id, run_name

In [83]:
'''mlflow log run start'''
exp_id, run_name = get_ml_logging_info()
mlflow.end_run()
mlflow.set_experiment(experiment_id=exp_id)
mlflow.start_run(run_name=run_name)

chain_type = "map_reduce"
# chain_type = "stuff"
# chain_type = "refine" 

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    # combine_docs_chain_kwargs={'prompt': reduce_prompt_template},
    # chain_type_kwargs={"map_prompt": map_prompt_template},
    return_source_documents=True,
    verbose=True,
    )

experiment with string id 2 is reused.


In [84]:
if chain_type == "map_reduce":
    qa_chain.combine_documents_chain.llm_chain.prompt = map_prompt_template
    qa_chain.combine_documents_chain.reduce_documents_chain.combine_documents_chain.llm_chain.prompt = reduce_prompt_template
    # set the token max from 3000 to 4000
    qa_chain.combine_documents_chain.reduce_documents_chain.token_max = MAX_POSITION_EMBEDDINGS
    
    
if chain_type == "refine":
    # pass
    qa_chain.combine_documents_chain.initial_llm_chain.prompt = init_prompt_template
    # qa_chain.combine_documents_chain.refine_llm_chain.token_max = MAX_POSITION_EMBEDDINGS

In [85]:
qa_chain

RetrievalQA(verbose=True, combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='<s>[INST] You are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don\'t know the answer to a question, please don\'t share false information. Just return "</s>"\n\n\nContext:\n{context}\n\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n\n[/INST]'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fabb0cefa90>, model_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'do_sample': True, 'top_k': 3, 'num_return_s

In [86]:
# qa_chain.combine_documents_chain.initial_llm_chain.prompt
# qa_chain.combine_documents_chain.refine_llm_chain.prompt

In [87]:
qa_chain

RetrievalQA(verbose=True, combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='<s>[INST] You are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don\'t know the answer to a question, please don\'t share false information. Just return "</s>"\n\n\nContext:\n{context}\n\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n\n[/INST]'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fabb0cefa90>, model_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'do_sample': True, 'top_k': 3, 'num_return_s

In [88]:
mlflow.log_param("map_prompt", map_prompt_template.template)
mlflow.log_param("user_query", query_template)
mlflow.log_param("doc_name", CUR_DOC_INFO.name)
mlflow.log_param("doc_source", CUR_DOC_INFO.source)
mlflow.log_param("llm_model", model_name)

if DEBUG:
    langchain.debug = True
response = qa_chain({"query": query})
if DEBUG:
    langchain.debug = False



> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/jovyan/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.


In [89]:
if DEBUG:
    print(f"Response: {response['result']}")
    print('-'*20)
    print(data[file_idx])

In [90]:
# use the string type
# age_schema = ResponseSchema(name="patient_age", description="patient age", type="int")
age_schema = ResponseSchema(name="patient_age", description="patient age")
# age_schema = ResponseSchema(name="patient_age", description="patient age", type="int")

# response_schema = [age_schema]
age_output_parser = StructuredOutputParser.from_response_schemas([age_schema])
# age_output_parser

In [91]:
age_question="retrieve one: patient age as number"
format_instructions = age_output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"patient_age": string  // patient age
}
```


In [92]:
input_text = response['result']

In [93]:
# chains patient name, log json parser chain instead of retrieval qa chain to anonymous patient name in experiment tracking logs
# mlflow.log_param("response_mapreduce", input_text)

In [94]:
if DEBUG:
    langchain.debug = True 
parser_response = chain.run(text=input_text, format_instructions=format_instructions, question=age_question, temperature=0.01)
if DEBUG:
    langchain.debug = False

mlflow.log_param("response_jsonchain", parser_response)
# end mlflow run logging
mlflow.end_run()

/home/jovyan/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [95]:
chain

LLMChain(prompt=PromptTemplate(input_variables=['format_instructions', 'question', 'text'], template="[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant.\nAlways answer as helpfully as possible using the context text provided.\nYour answers should only answer the question once and not have any text after the answer is done.\n\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\nIf you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\n\n\nCONTEXT:/n/n {text}/n/n/n\n\nQuestion: {question}/n\n{format_instructions}\n[/INST]"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fabb0cefa90>, model_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'do_sample': True, 'top_k': 3, 'num_return_sequences': 1, 'eos_token_id': 2, 'max_length': 4096, 'max_new_tokens': 80, 'temperature': 0.01, 'top_p': 0.8, '

In [96]:
# print(parser_response)

In [97]:
patient_age_obj = parse_response_dict(parser_response, age_output_parser, DEBUG).get("patient_age", "")
# print(patient_age_str)
try:
    if isinstance(patient_age_obj, str):
        patient_age_obj = patient_age_obj.strip()
        patient_age = int(patient_age_obj)
    if isinstance(patient_age_obj, int):
        patient_age = patient_age_obj
except Exception as e:
    print(e)
    patient_age = -1

if DEBUG:
    print(f"str response: {parser_response}")
    print(f"patient_age is: {patient_age}")
    print(f"pateint_age has type: {type(patient_age)}")

In [98]:
if DEBUG:
    print(input_text)
    print(parser_response)
    print(f"{patient_name} is {patient_age}")

In [99]:
# print(f"{patient_name} is {patient_age}")

In [100]:
# map_prompt_template = PromptTemplate(template="""
# Use the following portion of a long document to see if any of the text is relevant to answer the question. \n
# Return any relevant text verbatim.\n
# {context}\n
# Question: {question}\n
# Relevant text, if any:""",
# input_variables=['context', 'question'])
# qa_chain.combine_documents_chain.llm_chain.prompt = map_prompt_template

In [101]:
# stuff_llm_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n
# {context}\n\n
# Question: {question}\nHelpful Answer:"""

In [102]:
# qa_chain

In [103]:
# query = "What is the weight of the patient in kg? (Remember to include 'The weight of the patient is' in your answer.)"

In [104]:
# if DEBUG:
#     langchain.debug = True
# response = qa_chain({"query": query})
# #response = qa_chain({"query": query})
# if DEBUG:
#     langchain.debug = False

In [105]:
# if DEBUG:
#     print(f"Response: {response['result']}")
#     print('-'*20)
#     print(data[file_idx])

In [106]:
# response = index.query_with_sources(query, llm=llm, retriever_kwargs={"chain_type":"map_reduce"})

### (optional) Additional Read

GPT4All
* https://python.langchain.com/docs/integrations/llms/gpt4all

## Example prompt

In [107]:
context = ""

#### zero shot prompt

In [108]:
#name
input=f"Can you tell me the name of the patient from the folowing doctor's letter?\nLetter:\n{context}\nAnswer: "

In [109]:
#age
input=f"Can you tell me the age of the patient from the following doctor's letter?\nLetter:\n{context}\nAnswer: "

In [110]:
#diagnosis
input=f"Can you tell me the diagnosis of the patient from the following doctor's letter?\nLetter:\n{context}\nAnswer: "

#### Chain-of-thoughts prompt

In [111]:
# name prompt
input = f"Context: Patient: Fried\nQuestion: what is the name of the patient? \nAnswer: Name of the patient is Fried\nContext: {context}\nQuestion: what is the name of the patient?\nAnswer: the name of patient is"
#print(input)

In [112]:
# age prompt
input = f"Context:\nPatient: Fried is a 34-year-old patient\nQuestion:\nhow old is the patient? \nAnswer:\nFried is a patient, 34 year-old, the answers is 34\nContext:\n{context}\nQuestion:\nhow old is the patient?\nAnswer: "
# print(input)

In [113]:
# diagnose prompt
input=f"Context:\nPatient: Fried is a 34-year-old patient, Diagnoses: Influenza (J09.X2) \nQuestion:\nWhat diagnoses has the patient? \nAnswer:\nFried is a patient, 34 year-old, has diagnoses Influenza (J09.X2). The answers is Influenza (J09.X2)\nContext:\n{context}\nQuestion:\nWhat diagnoses has the patient?\nAnswer: "